## NOT successful attempt to solve Rossman store

### developped by 2 NUS students, A0175196U, A0174733B

#### This requires to instal the eras library to anaconda ! Keras allows us to build our neral networks, chose the number of hidden layers etc

In [2]:
import numpy as np
import pandas
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD
import numpy as np

In [3]:
store = pandas.read_csv('store.csv', sep = ',')
train_initial = pandas.read_csv('train_v2.csv',dtype={"StateHoliday": str})
test = pandas.read_csv('test_v2.csv', sep = ",") 

#take out the sales (we don't have this info in the test)
sales_train = train_initial['Sales']
train = train_initial.drop('Sales', axis=1)


#take out days were it's closed
sales_train = pandas.Series.to_frame(sales_train)
train = train[train.Customers != 0]
sales_train = sales_train[sales_train.Sales != 0]

In [4]:
# take out the stores with no sales - 
# train = train[train.Sales != 0] |||| NOT SURE IF GOOD IDEA

# put the 2 types of data together
df = train.join(store.set_index('Store'), on='Store')
test = test.join(store.set_index('Store'), on='Store')

# extract the date as day/month/year/weak of the year
df['Date']  = pandas.to_datetime(df['Date'], errors='coerce')
df['Year']  = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.weekofyear

test['Date']  = pandas.to_datetime(test['Date'], errors='coerce')
test['Year']  = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.weekofyear

In [5]:
# combine all to make sure test and data get the same dummies 
number_train = df.shape[0]
number_test = test.shape[0]
frames = [df,test]
result = pandas.concat(frames)

In [6]:
def month_difference (comp_year, our_year, comp_month, our_month):
    difference = our_month - comp_month + 12*(our_year-comp_year)
    return difference

result['CompetitionOpenSinceHowLong'] = month_difference (result['CompetitionOpenSinceYear'],result['Year'],
                                                       result['CompetitionOpenSinceMonth'],result['Month'])


In [7]:
#drop some of the feature

result.drop(['Store', 'CompetitionOpenSinceMonth', 'Promo2SinceWeek', 
             'PromoInterval','Date','Day'], axis=1, inplace=True)

In [8]:
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
result.StateHoliday.replace(mappings, inplace=True)
result.StoreType.replace(mappings, inplace=True)
result.Assortment.replace(mappings, inplace=True)

In [9]:
result = pandas.get_dummies(result, columns=['DayOfWeek', 'StateHoliday', 'StoreType', 
                                             'Assortment', 'Month', 'WeekOfYear', 'CompetitionOpenSinceYear', 
                                             'Promo2SinceYear', 'CompetitionOpenSinceHowLong' ], dummy_na = True)

In [10]:
# def days_between(d1, d2): #d1 and d2 must be datetime object
#     d1 = datetime.strptime(d1, "%Y-%m-%d")
#     d2 = datetime.strptime(d2, "%Y-%m-%d")
#     return abs((d2 - d1).days)

In [11]:
# def scalling (category) :
#     result[category] = result[category] - result[category].mean()
#     
# scalling ('Year')
# scalling('Customers')

# result['CompetitionDistance'] = result['CompetitionDistance'].fillna(0)
# 
# result['CompetitionFar'] = 0*result['CompetitionDistance']


        

        

In [12]:
# for i in range (result.shape[0]) :
#     if result['CompetitionExist'].iloc[i] > 0 :
#         result['CompetitionExist'].iloc[i] = 1
#     else :
#         result['CompetitionExist'].iloc[i] = 0
        
    # if result['CompetitionDistance'].iloc[i] > result['CompetitionDistance'].mean() :
    #     result['CompetitionFar'].iloc[i] = 1
    # else :
    #     result['CompetitionFar'].iloc[i] = 0

In [13]:
result.head()

,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Year,DayOfWeek_1.0,DayOfWeek_2.0,DayOfWeek_3.0,...,CompetitionOpenSinceHowLong_295.0,CompetitionOpenSinceHowLong_300.0,CompetitionOpenSinceHowLong_301.0,CompetitionOpenSinceHowLong_637.0,CompetitionOpenSinceHowLong_638.0,CompetitionOpenSinceHowLong_639.0,CompetitionOpenSinceHowLong_645.0,CompetitionOpenSinceHowLong_1380.0,CompetitionOpenSinceHowLong_1386.0,CompetitionOpenSinceHowLong_nan
0,616,1,1,0,1270.0,0,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,624,1,1,0,570.0,1,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,678,1,1,0,14130.0,1,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1632,1,1,0,620.0,0,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,617,1,1,0,29910.0,0,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df = result.iloc[:number_train,:]
test = result.iloc[number_train:,:]

In [15]:
df = df.astype(np.float32)
test = test.astype(np.float32)

In [16]:
df.head()

,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Year,DayOfWeek_1.0,DayOfWeek_2.0,DayOfWeek_3.0,...,CompetitionOpenSinceHowLong_295.0,CompetitionOpenSinceHowLong_300.0,CompetitionOpenSinceHowLong_301.0,CompetitionOpenSinceHowLong_637.0,CompetitionOpenSinceHowLong_638.0,CompetitionOpenSinceHowLong_639.0,CompetitionOpenSinceHowLong_645.0,CompetitionOpenSinceHowLong_1380.0,CompetitionOpenSinceHowLong_1386.0,CompetitionOpenSinceHowLong_nan
0,616.0,1.0,1.0,0.0,1270.0,0.0,2015.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,624.0,1.0,1.0,0.0,570.0,1.0,2015.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,678.0,1.0,1.0,0.0,14130.0,1.0,2015.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1632.0,1.0,1.0,0.0,620.0,0.0,2015.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,617.0,1.0,1.0,0.0,29910.0,0.0,2015.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#put the data as a numpy array
data_train = df.values
#sales_train = sales_train_train.values
sales_train = pandas.DataFrame.as_matrix(sales_train)

In [18]:
sales_train = sales_train.reshape(len(sales_train),)

In [19]:
data_train.shape

(54688, 307)

In [20]:
test_evaluate = test.values

In [21]:
# normalzing customers
#data_train[:,0]= (data_train[:,0] - data_train[:,0].mean()) / data_train[:,0].std( ddof=1)
# 

In [22]:
X = data_train
y = sales_train
y_standard_dev = y.std( ddof=1)
y_mean = y.mean()
# normalize label
y = (y - y_mean) / y_standard_dev

In [23]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.30)

In [31]:
# build sequential neural network model with:
# 150 hidden units
# 0.2 dropout regularizer
# tanh activation function
# mean squared error loss function
# stochastic gradient descent optimizer

model = Sequential()
model.add(Dense(70, input_dim=np.shape(X_train)[1], kernel_initializer='uniform'))
model.add(Activation('relu'))
model.add (Dropout(0.2))
model.add(Dense(20, kernel_initializer='uniform'))
model.add(Activation('relu'))
model.add (Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [32]:
history = model.fit(X_train, y_train, 
                    verbose=0, 
                    epochs=2000, 
                    batch_size=500,
                    validation_data=(X_validation, y_validation))

In [33]:
prediction_in_sample = model.evaluate(X_train, y_train)
print (prediction_in_sample)
ok = model.predict(X_train).flatten()*y_standard_dev + y_mean
print (ok)
#np.isnan(prediction_in_sample).sum()

37248/38281 [============================>.] - ETA: 0s[nan, 0.0]
[ nan  nan  nan ...,  nan  nan  nan]


In [ ]:
prediction_in_sample_validation = model.predict(X_validation)*y_standard_dev + y_mean

In [ ]:
prediction_out_sample = model.predict(test_evaluate)*y_standard_dev + y_mean

In [ ]:
for i in range (number_test) :
    if test['Customers'].iloc[i] == 0 :
        prediction_out_sample[i] = 0

In [ ]:
from math import sqrt
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y[i] != 0:
            summ += (1-y_pred[i]/y[i])**2
    return sqrt(summ/len(y))

In [ ]:
rmspe (y_train, prediction_in_sample)

In [ ]:
prediction_of_sales = prediction_out_sample

submission = pandas.DataFrame()
submission['Sales'] = prediction_of_sales
# submission.Sales = submission.Sales.astype(int)
cols = ['Id','Sales']
submission['Id'] = submission.index + 1
submission = submission[cols]
submission.to_csv('submission.csv', index=False)

In [ ]:
np.shape(X_train[1])